<a href="https://colab.research.google.com/github/jbenasuli/final_project/blob/database/amazon_product_selection_books.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [53.1 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Selected data
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
print((df.count(), len(df.columns)))

(3105520, 15)


In [ ]:
# Create a table to count
product_count = df.groupby("product_id").agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "reviews_count")
product_count.show()

+----------+-------------+
|product_id|reviews_count|
+----------+-------------+
|0825455189|            1|
|0590483595|            1|
|0842329129|         1321|
|0826472621|            1|
|0767905385|          200|
|0899573010|            1|
|1591472865|            2|
|0688075398|            3|
|1559502428|            2|
|0762404515|            2|
|1591840813|            4|
|0976538105|            1|
|1588513521|           27|
|1420837273|            5|
|0972437746|            8|
|1593113161|            1|
|0764228218|           20|
|155595250X|            1|
|0852345178|           36|
|1571202722|            1|
+----------+-------------+
only showing top 20 rows



In [ ]:
products_filtered = product_count.filter(product_count.reviews_count > 2000)
products_filtered.show()

+----------+-------------+
|product_id|reviews_count|
+----------+-------------+
|043935806X|         4625|
|0385504209|         2584|
|0895260174|         2615|
|0439139597|         3739|
|0316666343|         2024|
|0590353403|         2366|
|0439784549|         2052|
|0525947647|         2666|
+----------+-------------+

